In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import nilearn as nil
import os, re, glob, commands
import nipype.interfaces.fsl as fsl
from os.path import join

import nipype
from nipype.interfaces.spm import SliceTiming, Realign, Normalize12, Coregister, Smooth, Segment
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.algorithms.misc import Gunzip
from nipype.pipeline.engine import Workflow, Node
from nipype.interfaces.fsl import BET
import nipype.interfaces.spm as spm
import nipype.interfaces.fsl as fsl
from nipype.interfaces.fsl import maths
from nipype.pipeline.engine import Workflow, Node, MapNode

from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_matlab_cmd('/usr/local/matlab/R2016b/bin/matlab')

### variables

In [ ]:
experiment_dir = '/imaging/jerez/obi/hcp/'
data_dir = join(experiment_dir, 'data')

subject_list = ['114217','118023','119833','120111','120515','122418','123117','129028','130114','131823',
                '117021','118528','120010','120414','121416','122620','125222','130013','130417','133625']

# What is the name of your working directory and output folder
output_dir =  'output_firstSteps'
working_dir = 'workingdir_firstSteps'

# What are experiment specific parameters
number_of_slices = 72  
TR = 0.72                              
smoothing_size = 5
template = '/imaging/jerez/software/spm12/tpm/TPM.nii'

#### helper functions

In [ ]:
def plot_slice(fname):
    img = nib.load(fname)
    data = img.get_data()
    plt.imshow(np.rot90(data[:,:,int(data.shape[-1]/2)]),cmap='gray')
    plt.gca().set_axis_off()

### nodes

In [ ]:
# Gunzip - unzip functional
gunzip = MapNode(Gunzip(), iterfield = ['in_file'],name="gunzip")

# Slicetiming - correct for slice wise acquisition
interleaved_order = range(1,number_of_slices+1,2) + range(2,number_of_slices+1,2)
sliceTiming = Node(SliceTiming(num_slices=number_of_slices,
                               time_repetition=TR,
                               time_acquisition=TR-TR/number_of_slices,
                               slice_order=interleaved_order,
                               ref_slice=2),
                   name="sliceTiming")

# Realign - correct for motion
realign = Node(Realign(register_to_mean=True),
               name="realign")

# Artifact Detection - determine which of the images in the functional series
#   are outliers. This is based on deviation in intensity or movement.
art = Node(ArtifactDetect(norm_threshold=1,
                          zintensity_threshold=3,
                          mask_type='spm_global',
                          parameter_source='SPM'),
           name="art")

coregister = Node(Coregister(), name = 'coregister')
coregister.inputs.jobtype = 'estimate'

# Smooth - to smooth the images with a given kernel
smooth = Node(Smooth(fwhm=smoothing_size),
              name="smooth")

# Gunzip - unzip the structural image
gunzip_struct = Node(Gunzip(), name="gunzip_struct")

# Normalize - normalizes functional and structural images to the MNI template
normalize = Node(Normalize12(jobtype='estwrite',
                             tpm=template,
                             write_voxel_sizes=[1, 1, 1]),
                 name="normalize")

segment = Node(Segment(), name = 'segment')
segment.inputs.gm_output_type = [True, True, True]
segment.inputs.wm_output_type = [True, True, False]
segment.inputs.csf_output_type = [True, True, False]

#### Pipeline

In [ ]:
# Create a preprocessing workflow
preproc = Workflow(name='preproc')
preproc.base_dir = join(experiment_dir, working_dir)

# Connect all components of the preprocessing workflow
preproc.connect([(gunzip, sliceTiming, [('out_file', 'in_files')]),
                 (sliceTiming, realign, [('timecorrected_files', 'in_files')]),
                 (realign,coregister,[('mean_image', 'source'),
                                      #('realigned_files','apply_to_files')]),
                                      ('modified_in_files','apply_to_files')]),
                 (gunzip_struct, coregister, [('out_file', 'target')]),
                 (gunzip_struct, normalize, [('out_file', 'image_to_align')]),
                 (coregister, normalize, [('coregistered_files','apply_to_files')]),
                 (normalize, smooth, [('normalized_files', 'in_files')])
                 ])
                
#                  (gunzip_struct, normalize, [('out_file', 'image_to_align')]),
#                  (normalize, segment, [('normalized_files','data')])
#                  ])

In [ ]:
# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

# SelectFiles
templates = {'func': 'data/{subject_id}/unprocessed/3T/rfMRI_REST1_LR/{subject_id}_3T_rfMRI_REST1_LR.nii.gz',
             'anat': 'data/{subject_id}/T1w/T1w_acpc_dc.nii.gz'}
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                      name="selectfiles")

# Datasink
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id', '')]
datasink.inputs.substitutions = substitutions

# Connect SelectFiles and DataSink to the workflow
preproc.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                 (selectfiles, gunzip, [('func', 'in_file')]),
                 (selectfiles, gunzip_struct, [('anat', 'in_file')]),
                 (realign, datasink, [('mean_image', 'realign.@mean'),
                                      ('realignment_parameters',
                                       'realign.@parameters'),
                                      ]),
                 (normalize, datasink, [('normalized_files',
                                          'normalized.@files'),
                                         ('normalized_image',
                                          'normalized.@image'),
                                         ('deformation_field',
                                          'normalized.@field'),
                                         ]),
                 (smooth, datasink, [('smoothed_files', 'smoothed.@files')])
                 ])

In [ ]:
preproc.run('MultiProc', plugin_args={'n_procs': 6})